In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import csv

In [2]:
CHROME_DRIVER_PATH="D:\\00-env\\chromedriver.exe"
YOUTUBE_LINKS = {
#     'oh_my_girl': {
#         'nonstop' : 'https://www.youtube.com/watch?v=iDjQSdN_ig8',
#         'the_fifth_season' : 'https://www.youtube.com/watch?v=udGwca1HBM4',
#         'bungee' : 'https://www.youtube.com/watch?v=QTD_yleCK9Y',
#     },
#     'chungha': {
#         'stay_tonight': 'https://www.youtube.com/watch?v=YPFIh0dfYfw',
#         'roller_coaster': 'https://www.youtube.com/watch?v=900X9fDFLc4',
#         'snapping' : 'https://www.youtube.com/watch?v=deV_DmHKwjc'
#     },
#     'mamamoo': {
#         'spit_it_out': 'https://www.youtube.com/watch?v=bD06B-5SsSE',
#         'egotistic': 'https://www.youtube.com/watch?v=pHtxTSiPh5I',
#         'starry_night' : 'https://www.youtube.com/watch?v=0FB2EoKTK_Q'
#     },
#     'g_idle': {
#         'oh_my_god': 'https://www.youtube.com/watch?v=om3n2ni8luE',
#         'latata': 'https://www.youtube.com/watch?v=9mQk7Evt6Vs',
#         'hann' : 'https://www.youtube.com/watch?v=OKNXn2qCEws'
#     },
#     'itzy': {
#         'waanbe': 'https://www.youtube.com/watch?v=fE2h3lGlOsk',
#         'icy': 'https://www.youtube.com/watch?v=zndvqTc4P9I',
#         'dalla_dalla' : 'https://www.youtube.com/watch?v=pNfTK39k55U'
#     },
}

STORE_FILE_PATH= "D:\\02-yscec\\2020-1\\DataVisualization\\final-term\\data\\"

all_comments = dict()


In [4]:
def start_webpage(driver,link):
    driver.get(link)
    time.sleep(3)
    driver.execute_script("window.scrollTo(1, 500);")
    
def scrollDownToEnd(driver, pause_time=2):
    while True:
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script("window.scrollTo(1, document.documentElement.scrollHeight);")

        time.sleep(pause_time)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            driver.execute_script("window.scrollTo(1, document.documentElement.scrollHeight);")
            time.sleep(pause_time)

            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            else:
                continue

def print_dict(origin_key, d):
    for k, v in d.items():
        if isinstance(v, dict):
            print_dict(k, v)
        else:
            print(origin_key, k, v)

def get_all_comments(driver):
    comment_div= driver.find_element_by_xpath('//*[@id="contents"]')
    contents = comment_div.find_elements_by_xpath('//*[@id="content-text"]')
    authors = comment_div.find_elements_by_xpath('//*[@id="author-text"]')
    
    df = pd.DataFrame(columns=['author', 'contents'])
    all_comments = list()
    
    print("comments: ", len(contents))
    
    for idx, content in enumerate(contents):
        comment = dict()
        comment[authors[idx].text] = content.text
        all_comments.append(comment)
        
        df = df.append(pd.Series([authors[idx].text, content.text], index=df.columns), ignore_index=True)

    return df, all_comments

def automatic_crawlling(driver, singer, title, link):
    print(singer, title, link)
    start_webpage(driver, link)
    
    scrollDownToEnd(driver)
    
    df, comments = get_all_comments(driver)
    df.to_csv(STORE_FILE_PATH + singer + '_' + title + '.csv', mode='w', encoding='utf-8-sig')
    
    if singer not in all_comments.keys():
        all_comments[singer] = dict()
        
    all_comments[singer][title] = comments

def sequential_start(driver, d, origin_key=None):
    for k, v in d.items():
        if isinstance(v, dict):
            sequential_start(driver, v, k)
        else:
            automatic_crawlling(driver, origin_key, k, v)

In [ ]:
driver = webdriver.Chrome(CHROME_DRIVER_PATH)
sequential_start(driver, YOUTUBE_LINKS)

# Test

In [ ]:
singer = "oh_my_girl"
title = "nonstop"
driver = webdriver.Chrome(CHROME_DRIVER_PATH)
automatic_crawlling(webdriver.Chrome(CHROME_DRIVER_PATH), singer, title, YOUTUBE_LINKS[singer][title])

In [ ]:
driver = webdriver.Chrome(CHROME_DRIVER_PATH)
automatic_crawlling(driver, 'kangdaniel', '2u', 'https://www.youtube.com/watch?v=GmqfRBaJm6I')

kangdaniel 2u https://www.youtube.com/watch?v=GmqfRBaJm6I
comments:  1498
